In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
class data(Dataset):
  def __init__(self, file):
    self.file = pd.read_csv(file).to_numpy()

  def __len__(self):
     return self.file.shape[0]

  def __getitem__(self, idx):
      features = self.file[idx, :-1]
      label = self.file[idx, -1]
      return features, label

In [ ]:
dataset = data("water_potability.csv")

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=5, shuffle=False)
features, labels = next(iter(train_dataloader))

In [ ]:
import torch.nn as nn
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(9, 18)
    self.fc2 = nn.Linear(18, 6)
    self.fc3 = nn.Linear(6, 1)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = nn.functional.relu(self.fc1(x))
    x = self.dropout(x)
    x = nn.functional.relu(self.fc2(x))
    x = self.dropout(x)
    x = nn.functional.sigmoid(self.fc3(x))
    return x

model = NeuralNet()

In [ ]:
import torch.optim as optim
import numpy as np

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-5)

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

def compute_gradients():
    gradients = []
    for param in model.parameters():
        if param.grad is not None:
            gradients.append(param.grad.norm().item())
    return gradients

gradients_history = []

for epoch in range(100):
  epoch_train_loss = 0
  model.train()
  for features, labels in train_dataloader:
    optimizer.zero_grad()
    outputs = model(features.float())
    loss = criterion(outputs, labels.float().view(-1, 1))
    loss.backward()
    gradients = compute_gradients()
    gradients_history.append(gradients)
    optimizer.step()
    epoch_train_loss += loss.item()

  avg_train_loss = epoch_train_loss / len(train_dataloader)
  train_losses.append(avg_train_loss)


  model.eval()
  epoch_val_loss = 0
  val_acc = Accuracy(task = "binary")
  with torch.no_grad():
    for features, labels in val_dataloader:
      outputs = model(features.float())
      loss = criterion(outputs, labels.float(). view(-1, 1))
      epoch_val_loss += loss.item()

      predictions = (outputs >= 0.5).float()
      val_acc(predictions, labels.float().view(-1, 1))

  avg_val_loss = epoch_val_loss / len(val_dataloader)
  val_losses.append(avg_val_loss)

  accuracy = val_acc.compute()
  val_accuracies.append(accuracy)

  print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")

Epoch: 1, Training Loss: 0.5814967471527757, Validation Loss: 0.6100543608636032, Validation Accuracy: 0.6600496172904968
Epoch: 2, Training Loss: 0.5808926292735598, Validation Loss: 0.5982466283035867, Validation Accuracy: 0.6898263096809387
Epoch: 3, Training Loss: 0.5658968502875441, Validation Loss: 0.5994070865857748, Validation Accuracy: 0.692307710647583
Epoch: 4, Training Loss: 0.5793376640410897, Validation Loss: 0.6028707881032684, Validation Accuracy: 0.6749379634857178
Epoch: 5, Training Loss: 0.5724399444757041, Validation Loss: 0.6053879785316961, Validation Accuracy: 0.6699751615524292
Epoch: 6, Training Loss: 0.5889078454571481, Validation Loss: 0.603955959096367, Validation Accuracy: 0.6625310182571411
Epoch: 7, Training Loss: 0.5829154649136229, Validation Loss: 0.6109615660008089, Validation Accuracy: 0.6724565625190735
Epoch: 8, Training Loss: 0.5797067416010436, Validation Loss: 0.6004465906708328, Validation Accuracy: 0.6749379634857178
Epoch: 9, Training Loss: 0

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.show()

In [ ]:
# Plot gradient norms
plt.figure(figsize=(10, 5))
plt.plot(gradients_history)
plt.title('Gradient Norms Over Time')
plt.xlabel('Training Step')
plt.ylabel('Gradient Norm')
plt.show()